# NACHOS12-MAA01

First run with everything like EORCA12.L75-MJMgd16 without iceberg modules and modifs, ice shelf cavities, iceberg budget

## Compilation

 - NEMO version : NEMODRAK_4.0, svn co  -r 7046  http://forge.ipsl.jussieu.fr/nemo/svn/trunk/NEMOGCM
 - reference config : EORCA12.L75-MJMgd16
 - CPP.keys : key_trabbl, key_zdftke, key_zdftmx, key_lim3, key_mpp_mpi, key_iomput, key_xios2
 - modified routines
   * domzgr.F90 -> OPA_SRC/DOM/domzgr.F90  => ln_isfcav fixes pour bathy
   * istate.F90 -> OPA_SRC/DOM/istate.F90  => init tsb ?
   * ldfc1d_c2d.F90 -> OPA_SRC/LDF/ldfc1d_c2d.F90 => different handling of masks in diffusivity coefficient computation
   * lib_mpp.F90 -> OPA_SRC/LBC/lib_mpp.F90 => conditions for mmp_stop more drastic
   * sbcrnf.F90 -> OPA_SRC/SBC/sbcrnf.F90 => get rid of negative runoffs
   * zdftke.F90 -> OPA_SRC/ZDF/zdftke.F90 => behavior below sea ice



##  BDY

### from ZNATL12.L75-MJMgd16

  * [namelist](namelist.NACHOS12.L75-MAATEST), params :
    - CONFIG_IN=ZNATL12.L75
    - CASE_IN=MJMgd16
    - CONFIG_BDY=NACHOS12.L75
    - CASE_BDY=MAATEST
    - imin=5
    - jmin=65
    - years=2010-2010
    
  * fortran routine to mask of the nothern boundary : [mkmask](mkmask.f90), script : [make_mask_bdy_north.ksh](make_mask_bdy_north.ksh)
  
=> results : in /scratch/cnt0024/hmg2840/albert7a/NACHOS12.L75/NACHOS12.L75-I/BDY NACHOS12.L75_[N/S]bdy[SSH/TS/U/V]_y2010.nc

### from GLORYS2V3

  * on meolkerg (better from proxy reasons) download data from ftp CMEMS : [script](rapat_ftp_cmems.ksh) => results in /fsnet/data/meom/workdir/albert6a/GLORYS2V3/data/2010 and copied in /scratch/cnt0024/hmg2840/albert7a/GLORYS2V3/data/2010
  
   * cut the NACHOS region in GLORYS2V3 ncks -O -F -d x,757,1294 -d y,417,943  results => /fsnet/data/meom/workdir/albert6a/GLORYS2V3/data/2010/GLORYS2V3_NACHOS025 files downloaded on occigen at /scratch/cnt0024/hmg2840/albert7a/GLORYS2V3/data/2010

  * on occigen :

    * do a vertical extension of all data : [script](make_vertical_extent.ksh) thanks to JMM's tool ic_field_vertical_extent available via svn co https://servforge.legi.grenoble-inp.fr/svn/wg-drakkar/DATA_TOOLS/trunk/INITIAL_CONDITIONS => results in /scratch/cnt0024/hmg2840/albert7a/GLORYS2V3/data/2010
    * cut the proper area : [script](mk_bdydta.ksh)  => results in /scratch/cnt0024/hmg2840/albert7a/DEV/BDYGLORYS
    * repeat 3 times every columns in order to get a 1/12° resolution and mask according to NACHOS12.L75 (+ extrapolate some missing data) : [script](make_bdy_repeat_glorys.ksh) and [mktmask](mktmask.f90) => results in /scratch/cnt0024/hmg2840/albert7a/DEV/BDYGLORYS and copied in /scratch/cnt0024/hmg2840/albert7a/NACHOS12.L75/NACHOS12.L75-I/BDY
    
### from GLORYS2V3 cleaner version (we need the unclean version also)

  * modify the bathymetry at the frontier in order to fit the GLORYS one and repeat it on 3 j-level : [script](mk_modif_bathy.ksh) and [fortran routine](modif_bathy.f90) => result : /scratch/cnt0024/hmg2840/albert7a/NACHOS12.L75/NACHOS12.L75-I/NACHOS12.L75_bathymetry_20S_80N_nfrontiercleaned_repeat3.nc 
  
  * run NACHOS12.L75-MAA05 for a few steps in order to rebuil the new mask and meshs produced with this new bathymetry file, results : /scratch/cnt0024/hmg2840/albert7a/NACHOS12.L75/NACHOS12.L75-I/NACHOS12.L75_mask.nc NACHOS12.L75_mesh_hgr.nc NACHOS12.L75_mesh_zgr.nc
  
  * redo the previous steps (from GLORYS2V3) with the new mask
    

## Bathymetry and coordinates files

  * from [eORCA12_bathymetry_v2.4.nc](ftp://ftp.mercator-ocean.fr/download/users/rbourdal/AALBERT/eORCA12_bathymetry_v2.4.nc.gz) and [eORCA_R12_coordinates_v3.0.nc](ftp://ftp.mercator-ocean.fr/download/users/rbourdal/AALBERT/eORCA_R12_coordinates_v3.0.nc): cut ncks -F -d x,2271,3879 -d y,1807,3374
  
  * filling Lakes, Pacific Ocean, Indian Ocean, Red Sea, Black Sea, Hudson Bay, Baltique Sea, Northern Greenland Bays with bmgtools
  
=> results : in /scratch/cnt0024/hmg2840/albert7a/NACHOS12.L75/NACHOS12.L75-I NACHOS12.L75_coordinates_20S_80N.nc & NACHOS12.L75_bathymetry_20S_80N_nfrontiercleaned.nc

  * treatment of the bathymetry to fit GLORYS2V3 frontier (see BDY) => NACHOS12.L75_bathymetry_20S_80N_nfrontiercleaned_repeat3.nc

## MPP_PREP

on meolkerg, in /fsnet/data/meom/workdir/albert6a/NATL12/MPP_PREP

 *  run mpp_optimize with [namelist](namelist_MPP_PREP) => processor.layout
 *  ./screen.ksh 1560 => decompositions, the most optimal 49x61
 *  mpp_optimize.exe -keep 49 61
 *  ./plover_nc.ksh -f  NATL12-049x061_1560 => [NATL12-049x061_1560.gif](NATL12-049x061_1560.gif)

## Init files

### Every file from eORCA12 or ORCA12

  * [script](rapat_init_files.ksh)
  * from /scratch/cnt0024/hmg2840/molines/eORCA12.L75/eORCA12.L75-I files, cut ncks -F -d x,2271,3879 -d y,1807,3374
  * from /scratch/cnt0024/hmg2840/molines/ORCA12.L46/ORCA12.L46-I/ files, cut ncks -F -d x,2271,3879 -d y,1260,2827 
  * mask initial state with tmask from mask.nc [script](mkmaskinit.f90)
  * reflechir à dmp_mask : rappel baie hudson et entrée baltique
  
### Initial temperature, salinity and ice variables from GLORYS2V3

   * on meolkerg (better from proxy reasons) download data from ftp CMEMS : [script](rapat_ftp_cmems.ksh) => results in /fsnet/data/meom/workdir/albert6a/GLORYS2V3/data/2010 and copied in /scratch/cnt0024/hmg2840/albert7a/GLORYS2V3/data/2010

   * on meolkerg (better from proxy reasons) download data from ftp CMEMS : [script](rapat_glorys2v4.ksh) => results in /fsnet/data/meom/workdir/albert6a/GLORYS2V4/data and copied in /scratch/cnt0024/hmg2840/albert7a/GLORYS2V4/data

   * cut the NACHOS region in GLORYS2V3 ncks -O -F -d x,757,1294 -d y,417,943  results => /fsnet/data/meom/workdir/albert6a/GLORYS2V3/data/2010/GLORYS2V3_NACHOS025 files downloaded on occigen at /scratch/cnt0024/hmg2840/albert7a/GLORYS2V3/data/2010

   * cut the NACHOS region in GLORYS2V4 ncks -O -F -d x,756,1293 -d y,417,943  results => /scratch/cnt0024/hmg2840/albert7a/GLORYS2V4/data 

#### sosie interpolation method

* do a vertical extension of all data : [script](make_vertical_extent.ksh) thanks to JMM's tool ic_field_vertical_extent available via svn co https://servforge.legi.grenoble-inp.fr/svn/wg-drakkar/DATA_TOOLS/trunk/INITIAL_CONDITIONS => results in /scratch/cnt0024/hmg2840/albert7a/GLORYS2V3/data/2010
    
* interpolate the vertext with [sosie](https://github.com/brodeau/sosie) : [script](make_interp.ksh) with namelists [2D](namelist_2D) and [3D](namelist_3D) => very difficult to make it run, 15h job not always get in the queue, results GLORYS2V3-NACHOS12.L75_y2010 files not satisfying for 3D variable : irrealistic values north of the coasts at depth

#### cleaner method

 * do a vertical extension of all data : [script](make_vertical_extent.ksh) thanks to JMM's tool ic_field_vertical_extent available via svn co https://servforge.legi.grenoble-inp.fr/svn/wg-drakkar/DATA_TOOLS/trunk/INITIAL_CONDITIONS => results in /scratch/cnt0024/hmg2840/albert7a/GLORYS2V3/data/2010

 * drown the vertext files : [script](make_drown.ksh) with the mask_drown_field.x executable available with sosie tool, results concatenated in /scratch/cnt0024/hmg2840/albert7a/GLORYS2V3/data/2010/drowned_GLORYS2V3_NACHOS025_vertext_2010_gridT/S.nc
    
 * calculate the weigths converting NACHOS025 (=NARYS2V3) into NACHOS12
    
   * create pseudo coordinate files with regular grid for both NARYS2V3 and NACHOS12 : [script](mk_pseudo_coord.ksh) and [fortran routine](pseudo_coord.f90) 
    
   * in NACHOS12.L75-MAA05, run dcm_mktools.ksh -n WEIGHTS -m X64_OCCIGEN2jm2 -c NACHOS12.L75-MAA05 to compile the WEIGHTS tool copied from /scratch/cnt0024/hmg2840/albert7a/WNACHOS12.L75-MAA05/TOOLS to /scratch/cnt0024/hmg2840/albert7a/DEV/WEIGHTS
      
   * compute weights : [script](make_weights_narys-nachos12.ksh), results => wght_bilinear_NACHOS12_pseudo_coord.nc
   
#### even cleaner method

 * drown the GLORYS2V3 data without the vertical extension : [script](make_drown_novertext.ksh) with the mask_drown_field.x executable available with sosie tool => results /scratch/cnt0024/hmg2840/albert7a/GLORYS2V3/data/2010/drowned_GLORYS2V3_NACHOS025_2010_gridS/T.nc
 
 * replace the abnormal values with 9999 : [script](make_replace_abnormal_values.ksh) and [fortran routine](check_nan_and_replace.f90) => /scratch/cnt0024/hmg2840/albert7a/GLORYS2V3/data/2010/clean_drowned_GLORYS2V3_NACHOS025_2010_gridS/T.nc
 
 * do the vertical extension : [script](make_vertical_extent_clean_drowned.ksh) thanks to JMM's tool ic_field_vertical_extent available via svn co https://servforge.legi.grenoble-inp.fr/svn/wg-drakkar/DATA_TOOLS/trunk/INITIAL_CONDITIONS => results in /scratch/cnt0024/hmg2840/albert7a/GLORYS2V3/data/2010/vertext_clean_drowned_GLORYS2V3_NACHOS025_2010_gridS/T.nc
 
 * same weights than previously
   

## Running scripts

  * from EORCA12.L75-scal/CTL, [script](copy_files_for_run.ksh)